# 네이버 댓글 데이타 수집하기 

- 1 페이지 
- n 개 페이지 
- 전체 페이지 

In [18]:
# 관련모듈 임포트 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate

#https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1
#https://movie.naver.com/movie/point/af/list.naver?&page=1
12345
abcde

In [13]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'https://movie.naver.com/movie/point/af/list.naver?&page=1'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


<tr>
   <td class="ac num">18699331</td>
   <td class="title">
      <a href="/movie/bi/mi/basic.naver?code=167697" class="movie color_b">신과함께-인과 연</a>
         <div class="list_netizen_score">
            <span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
         </div>
         <br>지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음 
         <a href="#" onclick="report('oliv****', 'b302SgPzvFGwOd7jHwYDNYh2oahD4BHjDPjmxmV4xzk=', '지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음', '18699331', 'point_after');" class="report" style="color:#8F8F8F" title="새 창">신고</a>
   </td>
   <td class="num"><a href="javascript:find_list('nickname','18699331');" class="author">oliv****</a><br>23.01.23</td>
</tr>

In [14]:
# 2) 파싱 
# 영화제목, 평점, 댓글, 작성자, 날짜 
target_list = soup.select('tbody > tr')
print(len(target_list))
print(target_list[0])

10
<tr>
<td class="ac num">18736722</td>
<td class="title">
<a class="movie color_b" href="?st=mcode&amp;sword=187311&amp;target=after">바빌론</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:80%">별점 - 총 10점 중</span></span><em>8</em>
</div>
<br/>연출 잘했다 생각하지만 호불호는 갈릴듯 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('esun****', 'Ih4ep0PgSNdO8o0GBJwDQEZmiQ3cdbUwKheztsOxSx8=', '연출 잘했다 생각하지만 호불호는 갈릴듯', '18736722', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>
<td class="num"><a class="author" href="javascript:find_list('nickname','18736722');">esun****</a><br/>23.02.09</td>
</tr>


In [15]:
# 데이타 추출 
# 영화제목, 평점, 댓글, 작성자, 날짜 
print('제목 :', target_list[0].select_one('a').text)
print('평점 :', target_list[0].em.text)
# br 태그의 다음 요소로 접근
print('댓글 :', target_list[0].select_one('br').next_sibling.strip())
print('작성자 :', target_list[0].select_one('.author').text)
# td 태그 3번째 에서 br 태그 다음요소 접근
print('날짜 :', target_list[0].select('td')[2].select_one('br').next_sibling)

제목 : 바빌론
평점 : 8
댓글 : 연출 잘했다 생각하지만 호불호는 갈릴듯
작성자 : esun****
날짜 : 23.02.09


In [16]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
for tag in target_list:
    title = tag.select_one('a').text
    score = tag.em.text
    contents = tag.select_one('br').next_sibling.strip()
    writer = tag.select_one('.author').text
    date = tag.select('td')[2].select_one('br').next_sibling
    result_list.append([title, score, writer, date, contents])

In [17]:
# 4) 판다스의 데이타프레임 구조로 변경하기 
df = pd.DataFrame(result_list, 
                  columns=['영화 제목', '평점', '작성자','날짜', '댓글'])
df

,영화 제목,평점,작성자,날짜,댓글
0,바빌론,8,esun****,23.02.09,연출 잘했다 생각하지만 호불호는 갈릴듯
1,발신제한,1,sike****,23.02.09,암 유발 영화 전부 다 답답하네
2,남산의 부장들,1,leej****,23.02.09,좌깜 제대로 키셨네 불편한 영화다
3,남산의 부장들,1,leej****,23.02.09,좌깜 제대로 키셨네 불편한 영화다
4,타이타닉,10,eogh****,23.02.09,이 영화는 어렸을때보다 조금 자라고나서 보니 더 감동적이예요ㅠㅠㅠ
5,더 퍼스트 슬램덩크,10,sum1****,23.02.09,"대만아,,,,!!!"
6,더 퍼스트 슬램덩크,10,sdds****,23.02.09,여태 본 영화중 1등이네요
7,헤어질 결심,8,redo****,23.02.09,재밌고 쉽게 흘러가는 시간 속에 맺은 끝맺음에서 봉착한 난해함.
8,교섭,3,dldm****,23.02.09,스토리도 재미 없었는데 현빈님 연기를 너무 못해서... 아쉬웠어요. 보는데 계속 공...
9,다음 소희,10,love****,23.02.09,"픽션이 아님에 마음이 아팠던 영화, 세상에 꼭 존재해야 하는 류의 영화."


In [24]:
df_test = pd.DataFrame(result_list)
print(tabulate(df_test, headers='keys', tablefmt='plain', showindex=True))

    0                     1  2         3         4
 0  바빌론                8  esun****  23.02.09  연출 잘했다 생각하지만 호불호는 갈릴듯
 1  발신제한              1  sike****  23.02.09  암 유발 영화 전부 다 답답하네
 2  남산의 부장들         1  leej****  23.02.09  좌깜 제대로 키셨네 불편한 영화다
 3  남산의 부장들         1  leej****  23.02.09  좌깜 제대로 키셨네 불편한 영화다
 4  타이타닉             10  eogh****  23.02.09  이 영화는 어렸을때보다 조금 자라고나서 보니 더 감동적이예요ㅠㅠㅠ
 5  더 퍼스트 슬램덩크   10  sum1****  23.02.09  대만아,,,,!!!
 6  더 퍼스트 슬램덩크   10  sdds****  23.02.09  여태 본 영화중 1등이네요
 7  헤어질 결심           8  redo****  23.02.09  재밌고 쉽게 흘러가는 시간 속에 맺은 끝맺음에서 봉착한 난해함.
 8  교섭                  3  dldm****  23.02.09  스토리도 재미 없었는데 현빈님 연기를 너무 못해서... 아쉬웠어요. 보는데 계속 공조2 북한사람인줄 알았어요.
 9  다음 소희            10  love****  23.02.09  픽션이 아님에 마음이 아팠던 영화, 세상에 꼭 존재해야 하는 류의 영화.


In [26]:
base_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page='
# n개의 페이지 주소 
n = 10
result_list = []
for i in range(1, n+1):
    movie_url = base_url+str(i)
    res = requests.get(movie_url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    target_list = soup.select('tbody > tr')    
    for tag in target_list:
        title = tag.select_one('a').text
        score = tag.em.text
        contents = tag.select_one('br').next_sibling.strip()
        writer = tag.select_one('.author').text
        date = tag.select('td')[2].select_one('br').next_sibling
        result_list.append([title, score, writer, date, contents])
df = pd.DataFrame(result_list, 
    columns=['영화 제목', '평점', '작성자','날짜', '댓글'])
df    
  

,영화 제목,평점,작성자,날짜,댓글
0,신과함께-인과 연,10,oliv****,23.01.23,지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음
1,신과함께-인과 연,2,ckdg****,23.01.10,뇌절에 뇌절…뇌빼고만드신줄
2,신과함께-인과 연,7,greg****,23.01.08,음. 좋다 마른 영화 좀 별로...
3,신과함께-인과 연,10,kkin****,23.01.02,대박...이란말밖에
4,신과함께-인과 연,3,elec****,23.01.01,아무리 봐도 내 스타일의 영화는 아니다.
...,...,...,...,...,...
95,신과함께-인과 연,8,mks1****,21.11.06,재미있게 잘 봤어용.
96,신과함께-인과 연,10,buen****,21.11.03,1편에 비해 감동씬은 많지 않았지만 그냥 너무 잘 만든 느낌. 1편 보고 2편까지 ...
97,신과함께-인과 연,9,yujs****,21.10.25,"18.08.04.뭔가 감당 못할 스케일이라 난잡해질 듯 하였으나, 충분히 매력적이다..."
98,신과함께-인과 연,1,imrk****,21.10.15,


In [27]:
df.to_csv('data/gadwith.csv', index=False, encoding='utf-16' )

In [ ]:
- url : 키드 
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1

In [29]:
a = None
if a is None:
    print('None')
else:
    print('not none')

None


In [45]:
url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
print()

totla_page = int(soup.select_one('.c_88.fs_11').text)
print(totla_page)
page_num = totla_page // 10
if totla_page % 10 != 0:
    page_num +=1 
print(page_num)

for i in range(1, page_num+1):
    print(i)


146
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [32]:
# 페이지 주소 확인 
# 키드 = 13558
base_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
page = 1


while True:
    # 1) 주소 생성 
    movie_url = base_url+str(page)
    print(movie_url)
    
    # 2) 페이지 요청 후 숩 객체 생성 
    res = requests.get(movie_url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 3) 다음 버튼 파싱 
    next_btn = soup.select_one('.pg_next')  
    
    # 4) 마지막 페이지인지 검사
    if next_btn is None  :
        break
    else:    
    # 페이지 번호 증가     
        page += 1

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=10
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=